In [4]:
%matplotlib inline
import os
import glob
import json
from collections import Counter, defaultdict

import urllib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
# Get the data from the PPLM github repo (https://github.com/uber-research/PPLM)

files = ['clickbait.csv',
         'computers.csv',
         'legal.csv',
         'military.csv',
         'negative.csv',
         'politics.csv',
         'positive.csv',
         'religion.csv',
         'science.csv',
         'space.csv']

col_lists = []
dfs = {}
base_url = 'https://raw.githubusercontent.com/uber-research/PPLM/master/human_annotation/pplm_labeled_csvs/'
for infile in files:
    print(infile)
    # some files uses a header, others don't
    if infile == 'clickbait.csv' or infile == 'positive.csv' or infile == 'negative.csv':
        df = pd.read_csv(base_url + infile, header=1, index_col=None)
    else:
        df = pd.read_csv(base_url + infile, header=0, index_col=None)   
    col_lists.append(list(df.columns))
    dfs[infile] = df

for i in range(len(col_lists[0])):
    for cols in col_lists:
        print(cols[i], end=' ')
        print()    

clickbait.csv
computers.csv
legal.csv
military.csv
negative.csv
politics.csv
positive.csv
religion.csv
science.csv
space.csv
A 
From files: computers_pickles/computers_R.p, computers_pickles/computers_A.p, computers_pickles/computers_RB.p, computers_pickles/computers_A,B.p 
From files: legal_words_pickles/legal_words_R.p, legal_words_pickles/legal_words_A.p, legal_words_pickles/legal_words_RB.p, legal_words_pickles/legal_words_A,B.p 
From files: military_pickles/military_R.p, military_pickles/military_A.p, military_pickles/military_RB.p, military_pickles/military_A,B.p 
A 
From files: politics_pickles/politics_R.p, politics_pickles/politics_A.p, politics_pickles/politics_RB.p, politics_pickles/politics_A,B.p 
A 
From files: religion_pickles/religion_R.p, religion_pickles/religion_A.p, religion_pickles/religion_RB.p, religion_pickles/religion_A,B.p 
From files: science_pickles/science_R.p, science_pickles/science_A.p, science_pickles/science_RB.p, science_pickles/science_A,B.p 
From fil

In [9]:
df.head()

,"From files: space_pickles/space_R.p, space_pickles/space_A.p, space_pickles/space_RB.p, space_pickles/space_A,B.p",Unnamed: 1,Which passage is more topic relevant?,How fluent is the passage of A?,How fluent is the passage of B?,Ben,Which passage is more topic relevant?.1,How fluent is the passage of A?.1,How fluent is the passage of B?.1,North,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,<|endoftext|>It has been shown that a signific...,"<|endoftext|>It has been shown that, if we are...",B,2.0,2.0,NaN,B,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19_131_633
1,<|endoftext|>It has been shown that the amount...,<|endoftext|>It has been shown that in the hum...,neither,4.0,4.0,NaN,neither,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18_389_15
2,"<|endoftext|>To summarise, it was an interesti...","<|endoftext|>To summarise, this is a story abo...",neither,3.0,3.0,NaN,A,3.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_271_178
3,"<|endoftext|>Prior to this, the company was kn...","<|endoftext|>Prior to this, the company was kn...",B,3.0,3.0,NaN,B,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45_308_871
4,"<|endoftext|>More importantly, this is not an ...","<|endoftext|>More importantly, it would be the...",both,2.0,2.0,NaN,A,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10_751_705


In [13]:
# Look at the number of times each annotator appears
name_counter = Counter()
for infile in files:
    df = dfs[infile]
    columns = list(df.columns)        
    annotators = [columns[i].title() for i in [5, 9, 13]]
    name_counter.update(annotators)
for n, c in name_counter.most_common():
    print(n, c)

Gabe 4
Ben 4
Prabha 3
North 3
Unnamed: 13 2
Donnie 2
Shannon 2
Spencer 2
Ethan 2
Stephen 2
Raymis 1
Ally 1
Zakk 1
Hunter 1


In [14]:
# assign model IDs based on information in the dataset code/documentation
models = {0: 'B', 1: 'BC', 2: 'BR', 3: 'BCR'}
rows = []
text_counter = Counter()
for infile in files:
    df = dfs[infile]
    columns = list(df.columns)
    annotators = []
    for i in [5, 9, 13]:
        if columns[i].startswith('Unnamed'):
            name = 'Unnamed_' + basename
        else:
            name = columns[i].title()
        annotators.append(name)

    keys = df[columns[-1]].values
    ratings = df[[columns[i] for i in[3, 4, 7, 8, 11, 12]]].values
    texts_a = df[columns[0]].values
    texts_b = df[columns[1]].values
    
    for line_i, key in enumerate(keys):
        if type(key) == str:
            parts = key.split('_')
            model_a = models[int(parts[1]) % 4]
            model_b = models[int(parts[2]) % 4]
            text_a = texts_a[line_i]
            text_b = texts_b[line_i]
            text_counter.update([text_a])
            text_counter.update([text_b])    
            if not np.isnan(ratings[line_i, 0]):
                rows.append([text_a, model_a, annotators[0], ratings[line_i, 0]])
            if not np.isnan(ratings[line_i, 2]):
                rows.append([text_a, model_a, annotators[1], ratings[line_i, 2]])
            if not np.isnan(ratings[line_i, 4]):                
                rows.append([text_a, model_a, annotators[2], ratings[line_i, 4]])
            if not np.isnan(ratings[line_i, 1]):
                rows.append([text_b, model_b, annotators[0], ratings[line_i, 1]])
            if not np.isnan(ratings[line_i, 3]): 
                rows.append([text_b, model_b, annotators[1], ratings[line_i, 3]])
            if not np.isnan(ratings[line_i, 5]):    
                rows.append([text_b, model_b, annotators[2], ratings[line_i, 5]])
            
df_out = pd.DataFrame(rows, columns=['text', 'model', 'annotator', 'rating'])
df_out.head()

,text,model,annotator,rating
0,The food truck scene at SXSW in Austin this we...,BR,Gabe,3.0
1,The food truck scene at SXSW in Austin this we...,BR,Prabha,3.0
2,The food truck scene at SXSW in Austin this we...,BR,Unnamed_clickbait,4.0
3,The food and beverage industry is moving quick...,BC,Gabe,3.0
4,The food and beverage industry is moving quick...,BC,Prabha,4.0


In [15]:
ratings_per_text = defaultdict(list)
for row in rows:
    text = row[0]
    rating = row[-1]
    ratings_per_text[text].append(rating)
print(len(ratings_per_text))

1361


In [16]:
# convert everything to indices
text_counter = Counter(df_out['text'].values)
text_index = dict(zip(sorted(text_counter), range( len(text_counter))))
text_indices = ['I' + str(text_index[t]) for t in df_out['text'].values]
df_out['item_id'] = text_indices

model_counter = Counter(df_out['model'].values)
model_index = dict(zip(sorted(model_counter), range(len(model_counter))))
model_indices = [model_index[t] for t in df_out['model'].values]

annotator_counter = Counter(df_out['annotator'].values)
annotator_index = dict(zip(sorted(annotator_counter), range(len(annotator_counter))))
annotator_indices = ['W' + str(annotator_index[t]) for t in df_out['annotator'].values]
df_out['worker_id'] = annotator_indices

ratings = [float(r) for r in df_out['rating'].values]
print(len(text_indices), len(model_indices), len(annotator_indices), len(ratings))

rating_counter = Counter(ratings)
print(len(rating_counter))
df_out.head()

19486 19486 19486 19486
5


,text,model,annotator,rating,item_id,worker_id
0,The food truck scene at SXSW in Austin this we...,BR,Gabe,3.0,I1063,W4
1,The food truck scene at SXSW in Austin this we...,BR,Prabha,3.0,I1063,W7
2,The food truck scene at SXSW in Austin this we...,BR,Unnamed_clickbait,4.0,I1063,W12
3,The food and beverage industry is moving quick...,BC,Gabe,3.0,I1053,W4
4,The food and beverage industry is moving quick...,BC,Prabha,4.0,I1053,W7


In [17]:
print(set(df_out['model'].values))

{'BR', 'BC', 'BCR', 'B'}


In [18]:
print(set(df_out['rating']))

{1.0, 2.0, 3.0, 4.0, 5.0}


In [20]:
df_out['rating_scaled'] = [(r-1) / (len(rating_counter) - 1) for r in df_out['rating'].values]
df_out.head()

,text,model,annotator,rating,item_id,worker_id,rating_scaled
0,The food truck scene at SXSW in Austin this we...,BR,Gabe,3.0,I1063,W4,0.50
1,The food truck scene at SXSW in Austin this we...,BR,Prabha,3.0,I1063,W7,0.50
2,The food truck scene at SXSW in Austin this we...,BR,Unnamed_clickbait,4.0,I1063,W12,0.75
3,The food and beverage industry is moving quick...,BC,Gabe,3.0,I1053,W4,0.50
4,The food and beverage industry is moving quick...,BC,Prabha,4.0,I1053,W7,0.75


In [22]:
model_code_map = {'BR': 1, 'BC':2 , 'BCR': 3, 'B': 4}
model_codes = [model_code_map[m] for m in df_out['model'].values]
df_out['model_code'] = model_codes
df_out.head()

,text,model,annotator,rating,item_id,worker_id,rating_scaled,model_code
0,The food truck scene at SXSW in Austin this we...,BR,Gabe,3.0,I1063,W4,0.50,1
1,The food truck scene at SXSW in Austin this we...,BR,Prabha,3.0,I1063,W7,0.50,1
2,The food truck scene at SXSW in Austin this we...,BR,Unnamed_clickbait,4.0,I1063,W12,0.75,1
3,The food and beverage industry is moving quick...,BC,Gabe,3.0,I1053,W4,0.50,2
4,The food and beverage industry is moving quick...,BC,Prabha,4.0,I1053,W7,0.75,2


In [23]:
outdir = os.path.join('..', 'data', 'annotated_datasets')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df_out.to_csv(os.path.join(outdir, 'uber_all.csv'))